In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import time
from binarization import selected

In [6]:
def rescaleFrame(frame, scale = 0.75):
    height = int(frame.shape[0] * scale)
    width = int(frame.shape[1] * scale)

    return cv.resize(frame, (width, height), interpolation=cv.INTER_AREA)

In [7]:
def S_thresholder(S, thresh=(0, 255)):

    S_threshold = S * 0
    S_threshold[(S >= thresh[0]) & (S <= thresh[1])] = 1
    
    return S_threshold

In [9]:
# def abs_sobel_thresh(img, orient = 'x', sobel_kernel = 3, thresh = (0,255)):
#     if orient == "x":
#         sobel = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = sobel_kernel)
#     elif orient == "y":
#         sobel = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = sobel_kernel)


#     abs_sobel = np.abs(sobel)

#     scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))


#     sbinary = scaled_sobel * 0
#     sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel < thresh[1])] = 1

#     return sbinary

In [26]:
capture = cv.VideoCapture("Project_data/project_video.mp4")
time.sleep(2)

while (True):
    ret, frame = capture.read()

    wider_frame = frame * 0

    S = cv.cvtColor(frame, cv.COLOR_BGR2HLS)[:,:,2]

    sthresh = S_thresholder(S, (50, 225)) * 255

    # sobelx = abs_sobel_thresh(S, "x", 5, (40, 200)) * 255
    canny = cv.Canny(S, 50, 150)

    binary_sobx_Sthreshs = selected(frame)


    frame = rescaleFrame(frame, 0.5)

    sthresh = rescaleFrame(sthresh, 0.5)
    sthresh = cv.cvtColor(sthresh, cv.COLOR_GRAY2BGR)

    canny = rescaleFrame(canny, 0.5)
    canny = cv.cvtColor(canny, cv.COLOR_GRAY2BGR)

    binary_sobx_Sthreshs = rescaleFrame(binary_sobx_Sthreshs, 0.5)
    binary_sobx_Sthreshs = cv.cvtColor(binary_sobx_Sthreshs, cv.COLOR_GRAY2BGR)

    # [0,0]
    wider_frame[0:wider_frame.shape[0] // 2, 0 : wider_frame.shape[1] // 2] = frame
    # [0, 1]
    wider_frame[0:wider_frame.shape[0] // 2, wider_frame.shape[1] // 2 : wider_frame.shape[1]] = canny
    # [1, 0]
    wider_frame[wider_frame.shape[0] // 2: wider_frame.shape[0], 0 : wider_frame.shape[1] // 2] = sthresh
    # [1, 1]
    wider_frame[wider_frame.shape[0] // 2: wider_frame.shape[0], wider_frame.shape[1] // 2 : wider_frame.shape[1]] = binary_sobx_Sthreshs

    cv.imshow("Video", wider_frame)

    if cv.waitKey(20) & 0xFF == ord('q'):
        break
# cv.waitKey(0)
capture.release()
cv.destroyAllWindows()

In [16]:
binary_sobx_Sthreshs.max()

0

In [ ]:
capture = cv.VideoCapture("Project_data/project_video.mp4")
time.sleep(2)

while (True):
    ret, frame = capture.read()

    wider_frame = frame * 0


    S = cv.cvtColor(frame, cv.COLOR_BGR2HLS)[:,:,2]
    
    sthresh = S_thresholder(S, (45, 170)) * 255

    sthresh_eq = S_thresholder(cv.equalizeHist(S), (150, 220)) * 255
    


    frame = rescaleFrame(frame, 0.5)

    S = rescaleFrame(S, 0.5)
    S = cv.cvtColor(S, cv.COLOR_GRAY2BGR)

    sthresh = rescaleFrame(sthresh, 0.5)
    sthresh = cv.cvtColor(sthresh, cv.COLOR_GRAY2BGR)



    sthresh_eq = rescaleFrame(sthresh_eq, 0.5)
    sthresh_eq = cv.cvtColor(sthresh_eq, cv.COLOR_GRAY2BGR)

    wider_frame[0:wider_frame.shape[0] // 2, 0 : wider_frame.shape[1] // 2] = frame
    wider_frame[wider_frame.shape[0] // 2: wider_frame.shape[0], 0 : wider_frame.shape[1] // 2] = S
    wider_frame[0:wider_frame.shape[0] // 2, wider_frame.shape[1] // 2 : wider_frame.shape[1]] = sthresh
    wider_frame[wider_frame.shape[0] // 2: wider_frame.shape[0], wider_frame.shape[1] // 2 : wider_frame.shape[1]] = sthresh_eq

    cv.imshow("Video", wider_frame)

    if cv.waitKey(20) & 0xFF == ord('q'):
        break

capture.release()
cv.destroyAllWindows()